In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import math
import keras
from keras.layers import Dense,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from new_train_val_test import cc_train_val_test

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train,val,test = cc_train_val_test.get_upsampled_raw_dataset()

In [5]:
train

,clue,ans_len,ans,exp,is_anagram,is_homophone,is_double,is_cryptic,is_contain,is_reverse,is_alternate,is_init,is_delete,is_hidden,is_spoonerism,is_palindrome
39458,Electronic gadget linked with hacking,(6),KINDLE,Linked*,True,False,False,False,False,False,False,False,False,False,False,False
56310,Pitch one made strange as it sounds,(8),PHONETIC,An anagram (made strange) of PITCH ONE,True,False,False,False,False,False,False,False,False,False,False,False
14391,Craps game leads to slippery slope,(5),SCARP,CRAPS*,True,False,False,False,False,False,False,False,False,False,False,False
37371,Drag man out to aged parent,(7),GRANDMA,An anagram (‘out’) of ‘drag man’. The definit...,True,False,False,False,False,False,False,False,False,False,False,False
11703,Children's book laid out on rug,(7),MATILDA,MAT & LAID*,True,False,False,False,False,False,False,False,False,False,False,False
3380,Steers by intercepting one mobile,(4),OXEN,Ins of X (by as in 4 by 4 = 4 x 4) in *(ONE),True,False,False,False,False,False,False,False,False,False,False,False
64275,Got the better of new pee and pound,(3-5),ONE-UPPED,ONE-UPPED (PEE & POUND)* (*new),True,False,False,False,False,False,False,False,False,False,False,False
51037,Changed frock later on for one like Svengali?,"(7,5)",CONTROL FREAK,anagram (charged) of FROCK LATER ON,True,False,False,False,False,False,False,False,False,False,False,False
9051,"Lost fare in back of cab, say, showing a clean...",(8),BAREFOOT,"FARE* anagram=lost in BOOT (back of cab, say)",True,False,False,False,False,False,False,False,False,False,False,False
25226,Surprisingly civil or horribly bitter?,(7),VRIOLIC,VRIOLIC (vitriolic) Anagram (surprisingly) o...,True,False,False,False,False,False,False,False,False,False,False,False


In [4]:
train_x,train_y = train
val_x,val_y = val
test_x,test_y = test

In [5]:
filepath="./models/1xBilstm-{epoch:02d}-{loss:.2f}-{categorical_accuracy:.2f}-{val_loss:.2f}-{val_categorical_accuracy:.2f}-singlelabel.hdf5"
saveModelCallBack = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
from keras.callbacks import Callback

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

callbacks_list = [saveModelCallBack,tbCallBack,TestCallback((test_x, test_y))]


In [6]:
model = keras.Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 128))
model.add(Bidirectional(LSTM(128, dropout=0.5)))
model.add(Dense(12, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         5017216   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
Total params: 5,283,468
Trainable params: 5,283,468
Non-trainable params: 0
_________________________________________________________________


In [9]:
train_y.shape

(177792, 12)

In [10]:
test_y.shape

(9404, 12)

In [11]:
history = model.fit(train_x,train_y ,validation_data=(test_x,test_y), batch_size=128, epochs=15,shuffle=True,callbacks=callbacks_list,initial_epoch=1)


Train on 177792 samples, validate on 9404 samples
Epoch 2/15
177792/177792 [==============================] - 152s 856us/step - loss: 0.7835 - categorical_accuracy: 0.6847 - val_loss: 1.4898 - val_categorical_accuracy: 0.2738

Epoch 00002: saving model to ./models/1xBilstm-02-0.78-0.68-1.49-0.27-singlelabel.hdf5

Testing loss: 1.4898025237565748, acc: 0.2738196511868972

Epoch 3/15
 79744/177792 [============>.................] - ETA: 1:12 - loss: 0.2702 - categorical_accuracy: 0.8706

KeyboardInterrupt: 

In [ ]:
def simple_model():                                           
    model = keras.Sequential()
    model.add(Embedding(len(tokenizer.index_word)+1, 128,input_length=15))
    model.add(LSTM(128, dropout=0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [ ]:
model = simple_model()
model.fit(train_data,train_data_out,batch_size=1024)

In [239]:
model.evaluate(test_data,test_data_out)

3138/3138 [==============================] - 2s 482us/step


[1.9973295485616263, 0.18387507966857872]

In [293]:
from sklearn.ensemble import AdaBoostRegressor

In [294]:
from keras.wrappers.scikit_learn import KerasRegressor

In [295]:
bilstm_estimator = KerasRegressor(build_fn= simple_model, epochs=1, batch_size=1024, verbose=1)

In [297]:
import numpy as np
# train_data_out_labels = train_data_out.values.argmax(axis=1)
boosted_lstm = AdaBoostRegressor(base_estimator= bilstm_estimator)
boosted_lstm.fit(train_data, train_data_out)# scale your training data 

Epoch 1/1
28187/28187 [==============================] - 3s 112us/step
Epoch 1/1
28187/28187 [==============================] - 3s 116us/step
Epoch 1/1
28187/28187 [==============================] - 3s 119us/step
Epoch 1/1
28187/28187 [==============================] - 3s 121us/step
Epoch 1/1
28187/28187 [==============================] - 4s 127us/step
Epoch 1/1
28187/28187 [==============================] - 4s 126us/step
Epoch 1/1
28187/28187 [==============================] - 4s 125us/step
Epoch 1/1
28187/28187 [==============================] - 4s 130us/step
Epoch 1/1
28187/28187 [==============================] - 4s 131us/step
Epoch 1/1
28187/28187 [==============================] - 4s 131us/step


AdaBoostRegressor(base_estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x18a6db8d0>,
         learning_rate=1.0, loss='linear', n_estimators=50,
         random_state=None)

In [298]:
boosted_lstm.score(test_data,test_data_out)

3138/3138 [==============================] - 0s 74us/step


-3.974468899422262

In [215]:
train_data_out_labels = [np.argmax(row).astype(int) for row in train_data_out.values]

In [218]:
type(train_data_out_labels.astype(int))

AttributeError: 'list' object has no attribute 'astype'